In [1]:
from langchain_google_genai import GoogleGenerativeAI

import os
from dotenv import load_dotenv
load_dotenv() 

#llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=os.environ["GOOGLE_API_KEY"])
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=os.environ["GOOGLE_API_KEY"])

In [2]:
print(llm("write few lines on Generative AI"))

C:\Users\f\anaconda3\envs\env_langchain1\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Generative AI, a subset of artificial intelligence, has the remarkable ability to create new data, content, or experiences from existing data. Unlike traditional AI, which focuses on analyzing and interpreting data, generative AI can generate novel and original outputs, blurring the lines between human and machine creativity. This transformative technology has sparked excitement in various fields, including art, music, language processing, and drug discovery, opening up a world of possibilities for innovation and exploration.


#### Connect with database and ask some basic questions

In [3]:

#from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

from langchain.utilities import SQLDatabase


In [4]:
db_user = "root"
db_password = "root123"
db_host = "localhost"
db_name = "retail_sales_db"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)


print(db.table_info)


CREATE TABLE sales_tb (
	`TransactionID` INTEGER, 
	`Date` DATE, 
	`CustomerID` VARCHAR(10), 
	`Gender` VARCHAR(10), 
	`Age` INTEGER, 
	`ProductCategory` VARCHAR(50), 
	`Quantity` INTEGER, 
	`PriceperUnit` DECIMAL(10, 2), 
	`TotalAmount` DECIMAL(10, 2)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from sales_tb table:
TransactionID	Date	CustomerID	Gender	Age	ProductCategory	Quantity	PriceperUnit	TotalAmount
1	2023-11-24	CUST001	Male	34	Beauty	3	50.00	150.00
2	2023-02-27	CUST002	Female	26	Clothing	2	500.00	1000.00
3	2023-01-13	CUST003	Male	50	Electronics	1	30.00	30.00
*/


In [21]:
#Convert question to SQL query
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many customers are there"})
response

'```sql\nSELECT COUNT(DISTINCT `CustomerID`) AS `TotalCustomers`\nFROM sales_tb;\n```'

In [6]:
cleaned_query = response.strip('```sql\n').strip('\n```')
print(cleaned_query)

SELECT COUNT(DISTINCT `CustomerID`) AS `Number of Customers`
FROM sales_tb;


In [7]:
# Execute the cleaned query
result = db.run(cleaned_query)
print(result)

[(29,)]


In [14]:
chain = create_sql_query_chain(llm, db)
def execute_query(question):
    try:
        # Generate SQL query from question
        response = chain.invoke({"question": question})
        print(response)
        print("###################################################")
        # Strip the formatting markers from the response
        cleaned_query = response.strip('```sql\n').strip('\n```')
        print(cleaned_query)
        print("###################################################")        
        # Execute the cleaned query
        result = db.run(cleaned_query)
        print("###################################################")        
        # Display the result
        print(result)
    except ProgrammingError as e:
        print(f"An error occurred: {e}")


In [22]:
q1 = "How many unique customers are there for each product category"
execute_query(q1)

```sql
SELECT 
    `ProductCategory`, 
    COUNT(DISTINCT `CustomerID`) AS `UniqueCustomers`
FROM 
    `sales_tb`
GROUP BY 
    `ProductCategory`
ORDER BY 
    `ProductCategory`
LIMIT 
    5;
```
###################################################
SELECT 
    `ProductCategory`, 
    COUNT(DISTINCT `CustomerID`) AS `UniqueCustomers`
FROM 
    `sales_tb`
GROUP BY 
    `ProductCategory`
ORDER BY 
    `ProductCategory`
LIMIT 
    5;
###################################################
###################################################
[('Beauty', 8), ('Clothing', 13), ('Electronics', 8)]


In [16]:
q2 = "Calculate total sales amount per product category:"
execute_query(q2)

```sql
SELECT 
  `ProductCategory`, 
  SUM(`TotalAmount`) AS `TotalSalesAmount` 
FROM 
  `sales_tb` 
GROUP BY 
  `ProductCategory` 
ORDER BY 
  `TotalSalesAmount` DESC 
LIMIT 
  5;
```
###################################################
SELECT 
  `ProductCategory`, 
  SUM(`TotalAmount`) AS `TotalSalesAmount` 
FROM 
  `sales_tb` 
GROUP BY 
  `ProductCategory` 
ORDER BY 
  `TotalSalesAmount` DESC 
LIMIT 
  5;
###################################################
###################################################
[('Electronics', Decimal('5310.00')), ('Clothing', Decimal('5040.00')), ('Beauty', Decimal('1455.00'))]


In [17]:
q3 = "calculates the average age of customers grouped by gender."
execute_query(q3)

```sql
SELECT 
  `Gender`, 
  AVG(`Age`) AS `AverageAge`
FROM 
  `sales_tb`
GROUP BY 
  `Gender`
ORDER BY 
  `AverageAge` DESC
LIMIT 
  5;
```
###################################################
SELECT 
  `Gender`, 
  AVG(`Age`) AS `AverageAge`
FROM 
  `sales_tb`
GROUP BY 
  `Gender`
ORDER BY 
  `AverageAge` DESC
LIMIT 
  5;
###################################################
###################################################
[('Female', Decimal('43.3333')), ('Male', Decimal('35.2143'))]


In [18]:
q4 = "identify the top spending customers based on their total amount spent."
execute_query(q4)

```sql
SELECT 
    `CustomerID`, 
    SUM(`TotalAmount`) AS `TotalSpent`
FROM 
    `sales_tb`
GROUP BY 
    `CustomerID`
ORDER BY 
    `TotalSpent` DESC
LIMIT 
    5;
```
###################################################
SELECT 
    `CustomerID`, 
    SUM(`TotalAmount`) AS `TotalSpent`
FROM 
    `sales_tb`
GROUP BY 
    `CustomerID`
ORDER BY 
    `TotalSpent` DESC
LIMIT 
    5;
###################################################
###################################################
[('CUST015', Decimal('2000.00')), ('CUST013', Decimal('1500.00')), ('CUST016', Decimal('1500.00')), ('CUST002', Decimal('1000.00')), ('CUST026', Decimal('1000.00'))]


In [19]:
q5 = "counts the number of transactions made each month."
execute_query(q5)

```sql
SELECT
  strftime('%Y-%m', `Date`) AS `Month`,
  COUNT(*) AS `TransactionCount`
FROM sales_tb
GROUP BY `Month`
ORDER BY `TransactionCount` DESC
LIMIT 5;
```
###################################################
SELECT
  strftime('%Y-%m', `Date`) AS `Month`,
  COUNT(*) AS `TransactionCount`
FROM sales_tb
GROUP BY `Month`
ORDER BY `TransactionCount` DESC
LIMIT 5;
###################################################


NameError: name 'ProgrammingError' is not defined

In [20]:
q6 = "calculates the total sales amount and average price per unit for each product category."
execute_query(q6)

```sql
SELECT 
    `ProductCategory`, 
    SUM(`TotalAmount`) AS `TotalSalesAmount`, 
    AVG(`PriceperUnit`) AS `AveragePricePerUnit`
FROM 
    `sales_tb`
GROUP BY 
    `ProductCategory`
LIMIT 
    5;
```
###################################################
SELECT 
    `ProductCategory`, 
    SUM(`TotalAmount`) AS `TotalSalesAmount`, 
    AVG(`PriceperUnit`) AS `AveragePricePerUnit`
FROM 
    `sales_tb`
GROUP BY 
    `ProductCategory`
LIMIT 
    5;
###################################################
###################################################
[('Beauty', Decimal('1455.00'), Decimal('153.750000')), ('Clothing', Decimal('5040.00'), Decimal('183.461538')), ('Electronics', Decimal('5310.00'), Decimal('238.750000'))]
